In [1]:
!pip install pyspark



In [2]:
!pip install Koalas
!pip install bokeh


In [5]:
import numpy as np
import pandas as pd
from datetime import datetime
from bokeh.palettes import Spectral4 #4가지 선그래프 사용가능한 고정폼
from bokeh.plotting import figure, show, output_file,save
from bokeh.io import output_notebook, curdoc
from bokeh.models import Arc, ColumnDataSource, Grid, LinearAxis, Plot,LogAxis,Range1d,BoxAnnotation
from bokeh.themes import Theme

output_file(filename="승차-확진자 관련성.html", title="승차-확진자 관련성")
output_notebook()
curdoc().theme = Theme(filename="custom_bokeh_theme.yml")
format = '%Y-%m-%d'


Loading BokehJS ...

In [2]:
bb = pd.read_parquet('second.parquet/confirmed_cases.parquet')
cc = pd.read_parquet('second.parquet/metro_per_week.parquet')
#폼에 넣기 위해 컬럼명 변경
cc.rename(columns={'사용일자':'주간시작일자'},inplace =True)
cc.rename(columns={'승차총승객수':'신규확진자'},inplace =True)

def fun(x):
    return x*2000 + 22000000

bb["신규확진자"] = bb["신규확진자"].apply(fun)

        주간시작일자           일자   누적확진자     신규확진자
0   2021-09-20  09/20-09/26  303553  54034000
1   2021-09-13  09/13-09/19  287536  48242000
2   2021-09-06  09/06-09/12  274415  47274000
3   2021-08-30  08/30-09/05  261778  45454000
4   2021-08-23  08/23-08/29  250051  46538000
..         ...          ...     ...       ...
83  2020-02-17  02/17-02/23     556  23054000
84  2020-02-10  02/10-02/16      29  22004000
85  2020-02-03  02/03-02/09      27  22024000
86  2020-01-27  01/27-02/02      15  22024000
87  2020-01-20  01/20-01/26       3  22006000

[88 rows x 4 columns]
        주간시작일자     신규확진자
0   2020-01-06  51210906
1   2020-01-13  51766720
2   2020-01-20  37274813
3   2020-01-27  45239198
4   2020-02-03  43106598
..         ...       ...
85  2021-08-23  37077195
86  2021-08-30  38017487
87  2021-09-06  38915705
88  2021-09-13  35178269
89  2021-09-20  30632894

[90 rows x 2 columns]


In [6]:

p = figure(width=800, height=500, x_axis_type="datetime")

p.extra_y_ranges = {"foo": Range1d(start=0, end=17000)}
p.add_layout(LinearAxis(y_range_name="foo"), 'right')
    
p.title.text = '승차-확진자 관련성'
zips = zip([bb, cc], ["주간산규확진자", "승차인원"], Spectral4)

for data, name, color in zip([bb, cc], ["주간산규확진자", "승차인원"], Spectral4):
    df = pd.DataFrame(data)
    df['주간시작일자'] = pd.to_datetime(df['주간시작일자'])
    p.line(df['주간시작일자'], df['신규확진자'], line_width=2, color=color, alpha=1,
           muted_color=color, muted_alpha=0.2, legend_label=name)
#박스를 적용 해봅시다
"""20.3.22~5.5	5단계(사회적거리두기)
5.6~8.15	1단계
8.16~8.29	3단계
8.16~9.13	4단계
9.14~10.11	3단계
10.12~11.23	1단계
11.24~12.7	3단계
12.8~21.2.14	4단계
2.15~7.11	3단계"""

def date(strs):
    return datetime.strptime(strs,format)

box = BoxAnnotation(left=date("2020-03-22"), right=date("2020-05-05"),fill_alpha=0.45, fill_color='red')
p.add_layout(box)
box = BoxAnnotation(left=date("2020-05-05"), right=date("2020-08-15"),fill_alpha=0.18, fill_color='red')
p.add_layout(box)
box = BoxAnnotation(left=date("2020-08-15"), right=date("2020-08-29"),fill_alpha=0.3, fill_color='red')
p.add_layout(box)
box = BoxAnnotation(left=date("2020-08-29"), right=date("2020-09-13"),fill_alpha=0.4, fill_color='red')
p.add_layout(box)
box = BoxAnnotation(left=date("2020-09-13"), right=date("2020-10-11"),fill_alpha=0.3, fill_color='red')
p.add_layout(box)
box = BoxAnnotation(left=date("2020-10-11"), right=date("2020-11-23"),fill_alpha=0.18, fill_color='red')
p.add_layout(box)
box = BoxAnnotation(left=date("2020-11-23"), right=date("2020-12-07"),fill_alpha=0.3, fill_color='red')
p.add_layout(box)
box = BoxAnnotation(left=date("2020-12-07"), right=date("2021-02-14"),fill_alpha=0.4, fill_color='red')
p.add_layout(box)
box = BoxAnnotation(left=date("2021-02-14"), right=date("2021-09-20"),fill_alpha=0.3, fill_color='red')
p.add_layout(box)
#인터랙트 위젯 적용
p.legend.location = "top_left"
p.legend.click_policy="mute"

show(p)